# Q学習

Q学習を実装してみる

In [12]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display, HTML
from tqdm import tqdm


---

## CartPole

今回扱う問題．ポールが倒れないようなカート操作ができるように学習させる．

- [Cart Pole - Gymnasium Documentation](https://gymnasium.farama.org/environments/classic_control/cart_pole/)

![](https://gymnasium.farama.org/_images/cart_pole.gif)

<br>

以下の4つの状態を持つ

| 状態 | 範囲 |
| --- | --- |
|カートの位置 | -4.8 ~ 4.8 |
| カートの速度 | -Inf ~ Inf |
| ポールの角度 | -24° ~ 24° |
| ポールの角速度 | -Inf ~ Inf |

また行動はカートを右に動かすか左に動かすかの2通り．

In [13]:
ENV = 'CartPole-v1'
env = gym.make(ENV, render_mode='rgb_array')
n_states = env.observation_space.shape[0]
n_actions = env.action_space.n

print('n_actions:', n_actions)
print('n_states:', n_states)

n_actions: 2
n_states: 4



---

## Q学習

### 状態の離散化

Q学習では状態を離散化する必要がある．  
今回は全ての状態を6段階に分割する．つまり状態は$6^4=1296$通りになる．

In [3]:
NUM_DIZITIZED = 6

def make_bins(min, max, n=NUM_DIZITIZED):
    return np.linspace(min, max, n+1)[1:-1]

def digitize_state(state: np.ndarray) -> int:
    """状態を離散化する"""
    cart_p, cart_v, pole_a, pole_v = state
    digi_cart_p = np.digitize(cart_p, make_bins(-4.8, 4.8))
    digi_cart_v = np.digitize(cart_v, make_bins(-4.8, 4.8))
    digi_pole_a = np.digitize(pole_a, make_bins(-0.24, 0.24))
    digi_pole_v = np.digitize(pole_v, make_bins(-4.8, 4.8))
    digi = sum([NUM_DIZITIZED**i * d for i, d in enumerate([
        digi_cart_p, digi_cart_v, digi_pole_a, digi_pole_v])])
    return digi

### エージェント

Q関数を所持し，それを元に行動を決定できるエージェントをクラスとして実装する．

In [4]:
class Agent:
    def __init__(self,):
        self.q = np.random.randn(NUM_DIZITIZED**4, n_actions) # Q関数の初期化

    def get_action(self, s, epsilon=0):
        s = digitize_state(s) # 状態を離散化
        if np.random.random() < epsilon: # ε-greedy法
            a = np.random.randint(n_actions)
        else:
            a = np.argmax(self.q[s])
        return a

    def update(self, s, a, r, next_s, gamma, alpha):
        """Q関数を更新する"""
        s = digitize_state(s)
        next_s = digitize_state(next_s)
        target = r + gamma * np.max(self.q[next_s])
        self.q[s, a] = self.q[s, a] + alpha * (target - self.q[s, a])

### 報酬

報酬は，ポールの角度の絶対値にマイナスをかけたものとする．  
ポールの角度が0°に近いほど，報酬は大きくなる．

In [5]:
def reward_func(s):
    cart_p, cart_v, pole_a, pole_v = s
    r = -abs(pole_a)
    return r

### 描画

ゲーム画面を描画する関数も実装

In [6]:
def run(agent, env, lim=500, interval=50):
    frames = []
    s, _ = env.reset()
    done = False
    for _ in range(lim):
        a = agent.get_action(s)
        s, _, done, _, _ = env.step(a)
        frames.append(env.render())
        if done:
            break

    fig = plt.figure()
    plt.axis('off')
    im = plt.imshow(frames[0])

    def update(i):
        im.set_array(frames[i])
        return im,

    ani = animation.FuncAnimation(
        fig, update, frames=len(frames), interval=interval)
    plt.close()
    display(HTML(ani.to_jshtml()))

### 学習

学習を行う関数の実装．  
行動決定→行動→状態遷移→報酬決定→Q関数更新 を繰り返す

In [7]:
def train(env, agent, n_episodes, epsilon=0.2, gamma=0.9, alpha=0.3):
    for _ in tqdm(range(n_episodes)):
        s, _ = env.reset()
        done = False
        while not done:
            a = agent.get_action(s, epsilon)
            next_s, _, done, _, _ = env.step(a)
            r = reward_func(next_s)
            agent.update(s, a, r, next_s, gamma, alpha)
            s = next_s

実際に学習させてみる．まずエージェント（Q関数）を初期化

In [8]:
agent = Agent()

初期状態での性能はこんな感じ

In [9]:
run(agent, env)

/Users/komiya/Private/study/reinforcement-learning/.venv/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


ここから学習させる．2000エピソード

In [10]:
train(env, agent, 2000)

100%|██████████| 2000/2000 [00:11<00:00, 177.01it/s]


学習結果

In [11]:
run(agent, env)